In [1]:
from src.tools import to_ranges, count_im, xy
from matplotlib import pyplot as plt
import numpy as np
import cv2 as C

BEAD_FREQ = 54
B_POS = (148, 710)
B_SIZ = 25

BEAD_SIZE = 7 #px
INTRA_BEAD_DIST = 3 #in terms of R

hough = lambda img, **kwargs: np.uint16(np.around(C.HoughCircles(img, **kwargs)))

In [2]:
INTRA_BEAD_DIST = (INTRA_BEAD_DIST + 2)*BEAD_SIZE

B_POS = (B_POS[0], B_POS[1] - INTRA_BEAD_DIST)
x0, y0, x1, y1 = xy(B_POS, B_SIZ)

flen = count_im('out_dir')
print(f"Found {flen} images")

Found 98 images


In [3]:
lums = []
cache = {}

for i in range(1, flen + 1):
  name = f'{i:03d}'
  img = C.imread(f'./out_dir/{name}.png', C.IMREAD_GRAYSCALE)

  # mean lum s.t. lum = sum_white (>200)
  lum = np.sum(img > 200)
  lums.append(lum)
  cache[name] = lum
mean, std = np.mean(lums), np.std(lums)

# delete items in cache that are not in lums
for k in list(cache.keys()):
  if cache[k] < mean + std:
    del cache[k]

ranges = to_ranges(list(map(int, cache.keys())))
print(ranges)

[range(23, 27), range(74, 81)]


In [4]:
# for each range, find the argmax
maxes = {}
for r in ranges:
  argmax = max(r, key=lambda k: cache[f"{k:03d}"])
  maxes[argmax] = cache[f"{argmax:03d}"]

# max in maxes contains the bead
bead = max(maxes, key=lambda k: maxes[k])
img_base = C.imread(f'./in_dir/{bead:03d}.png', C.IMREAD_GRAYSCALE)
img_bead = C.imread(f'./in_dir/{bead - BEAD_FREQ:03d}.png', C.IMREAD_GRAYSCALE)

img_base = img_base[x0:x1, y0:y1]
img_bead = img_bead[x0:x1, y0:y1]

In [5]:
detected = hough(
  edges, method=C.HOUGH_GRADIENT, dp=1,
  minDist=img.shape[0] / (2 * (R + E)), param1=100, param2=10,
  minRadius=R - E, maxRadius=R + E
)

NameError: name 'edges' is not defined